# OOP solution

In [1]:
import numpy as np

# lines and rows only have unique values
def lineCheck(line):
    line = [i for i in line if i > 0]
    res = len(line) == len(np.unique(line))
    return res

# all numbers must be between 0 and 9
def valueCheck(line):
    line = list(line)
    checkVal = min(line) >=0 and max(line) <=9
    return checkVal

# a sub-grid (3X3) only has unique values
def checkSquare(grid):
    grid = np.ndarray.flatten(grid)
    res = lineCheck(grid)
    return res 

# wrapper to call all the the check funcitons
def validateGrid(grid):
    if not grid.shape == (9,9):
        errMsg = 'Invalid grid. Must be 9X9'
        return False, errMsg
    if not grid.dtype == 'int32':
        errMsg = 'Invalid grid. All elements in the grid must be integers.'
        return False, errMsg
    rows = all(np.apply_along_axis(lineCheck, 0, grid))
    cols = all(np.apply_along_axis(lineCheck, 1, grid))
    if( not rows or not cols):
        errMsg = 'Invalid grid. Duplicates in rows or columns.'
        return False, errMsg
    vals = all(np.apply_along_axis(valueCheck, 1, grid))
    if( not vals):
        errMsg = 'Invalid grid. All numbers must be between 0 and 9.'
        return False, errMsg
    gridCoord = [0,3,6]
    checkSquareRes = []
    for i in gridCoord:
        for ii in gridCoord:
            square = grid[i:i+3, ii:ii+3]
            checkSquareRes = np.append(checkSquareRes, checkSquare(square))
            
    
    checkSquareRes = all(checkSquareRes)
    if not checkSquareRes:
        errMsg = 'Invalid grid. Duplicates in a square.'
        return False, errMsg
    
    return True, None

def possible(y, x, n, grid):
    """
    function to check if we can put a number in a (x, y) place of grid
    if we can put the number in the x/y space return True, False otherwise
    """
    # check if the number appears in the row
    for i in range(0,9):
        if grid[y][i] == n:
            return False
    # check if the number appears in the column
    for i in range(0,9):
        if grid[i][x] == n:
            return False
    # check if the number appears in a 3X3 square
    x0 = (x//3)*3
    y0 = (y//3)*3
    for i in range(0,3):
        for j in range(0,3):
            if grid[y0+i][x0+j] == n:
                return False
    # if we can put the number there
    return True

class sudoku:
    def __init__(self, grid):
        isValidGrid, errMsg = validateGrid(np.array(grid))
        if isValidGrid:
            self.grid = grid
        else:
            print(errMsg)
        
    # nice way to print the grid
    def print_grid(self):
        if not hasattr(self, 'grid'):
            print('You need to provide a grid to print first.')
            return
        for line in self.grid:
            for square in line:
                if square == 0:
                    # replace 0 with ".""
                    print('.', end = ' ')
                else:
                    print(square, end = ' ')
            print()
        print('\n')
    
    def solve_puzzle(self):
        if not hasattr(self, 'grid'):
            print('You need to provide a grid to solve first.')
            return
        # loop through rows and columns
        for y in range(0,9):
            for x in range(0,9):
                if self.grid[y][x] == 0:
                    # if the value is 0, we try to put a number
                    # brute force approach by simply try all the numbers
                    for n in range(1,10):
                        if possible(y,x,n,self.grid):
                            # it is possible to put the number: replace the 0 with the new value
                            self.grid[y][x] = n
                            # now we can call solve again, since we have simplyfied the puzzle
                            self.solve_puzzle()
                            #stop the recursive loop and give the first valid answer
                            if np.count_nonzero(np.array(self.grid)) == 81:
                                return self.grid
                            # answer not found. backtracking
                            self.grid[y][x] = 0
                    # we could not solve the puzzle    
                    if x==8 and y==8 and n==9:
                        # we have tried everything
                        print('This puzzle has no solutions.')
                    # we can still backtrack
                    return 
        
        return

Usage example

In [2]:
obj = sudoku([[5,3,0,0,7,0,0,0,0],
              [6,0,0,1,9,5,0,0,0],
              [0,9,8,0,0,0,0,6,0],
              [8,0,0,0,6,0,0,0,3],
              [4,0,0,8,0,3,0,0,1],
              [7,0,0,0,2,0,0,0,6],
              [0,6,0,0,0,0,2,8,0],
              [0,0,0,4,1,9,0,0,5],
              [0,0,0,0,8,0,0,7,9]])
obj.solve_puzzle()
obj.print_grid()

5 3 4 6 7 8 9 1 2 
6 7 2 1 9 5 3 4 8 
1 9 8 3 4 2 5 6 7 
8 5 9 7 6 1 4 2 3 
4 2 6 8 5 3 7 9 1 
7 1 3 9 2 4 8 5 6 
9 6 1 5 3 7 2 8 4 
2 8 7 4 1 9 6 3 5 
3 4 5 2 8 6 1 7 9 




# Unit testing 

In [3]:
import unittest

class validations(unittest.TestCase):
    # test the grid validation 
    def test_lineCheck(self):
        # ok line
        self.assertTrue(lineCheck([1,2,3,4]))
        # ok line. 0s do not count
        self.assertTrue(lineCheck([1,2,3,4, 0, 0]))
        # Not ok. duplicated 1
        self.assertFalse(lineCheck([1,2,3,4, 1, 0]))
    # test all numbers are between 0 and 9
    def test_valueCheck(self):
        # ok case
        self.assertTrue(valueCheck([1,2,3,4,0]))
        # no, we have a <0
        self.assertFalse(valueCheck([1,2,3,4,-1]))
        # no, we have a >10
        self.assertFalse(valueCheck([1,2,3,4,1,10]))
    # check that a 3X3 square is valid
    # the function execute the test by calling checkLine
    # which has been tested above
    def test_checkSquare(self):
        # ok case
        self.assertTrue(checkSquare(np.array([[1,2,3],
                                              [4,5,6],
                                              [7,8,9]])))
        # not ok, duplicated 1
        self.assertFalse(checkSquare(np.array([[1,2,3],
                                              [4,5,6],
                                              [7,8,1]])))
        # ok, 0s do not count
        self.assertTrue(checkSquare(np.array([[1,2,3],
                                              [4,0,6],
                                              [7,8,0]])))
    # the function checking the grid overall.
    # this function alkso calls cjekcLine and checkSquare
    def test_validateGrid(self):
        # ok case
        self.assertTrue(validateGrid(np.array([[5,3,0,0,7,0,0,0,0],
                                               [6,0,0,1,9,5,0,0,0],
                                               [0,9,8,0,0,0,0,6,0],
                                               [8,0,0,0,6,0,0,0,3],
                                               [4,0,0,8,0,3,0,0,1],
                                               [7,0,0,0,2,0,0,0,6],
                                               [0,6,0,0,0,0,2,8,0],
                                               [0,0,0,4,1,9,0,0,5],
                                               [0,0,0,0,8,0,0,7,9]])))
        # not ok: duplicated 9 in next to last column
        self.assertFalse(validateGrid(np.array([[5,3,4,6,7,8,9,1,2],
                                                [6,7,2,1,9,5,3,4,8],
                                                [1,9,8,3,4,2,5,6,7],
                                                [8,5,9,7,6,1,4,2,3],
                                                [4,2,6,8,5,3,7,9,1],
                                                [7,1,3,9,2,4,8,5,6],
                                                [9,6,1,5,3,7,2,8,4],
                                                [2,8,7,4,1,9,6,3,5],
                                                [3,4,5,2,8,6,1,9,0]]))[0])
        # not ok: it has non numeric value
        self.assertFalse(validateGrid(np.array([[5,3,4,6,7,8,9,1,2],
                                                [6,7,2,1,9,5,3,4,8],
                                                [1,9,8,3,4,2,5,6,7],
                                                [8,5,9,7,6,1,4,2,3],
                                                [4,2,6,8,5,3,7,9,1],
                                                [7,1,3,9,2,4,8,5,6],
                                                [9,6,1,5,3,7,2,8,4],
                                                [2,8,7,4,1,9,6,3,5],
                                                [3,4,5,2,8,6,1,7,'9']]))[0])
        # not ok: the last number is >9
        self.assertFalse(validateGrid(np.array([[5,3,4,6,7,8,9,1,2],
                                                [6,7,2,1,9,5,3,4,8],
                                                [1,9,8,3,4,2,5,6,7],
                                                [8,5,9,7,6,1,4,2,3],
                                                [4,2,6,8,5,3,7,9,1],
                                                [7,1,3,9,2,4,8,5,6],
                                                [9,6,1,5,3,7,2,8,4],
                                                [2,8,7,4,1,9,6,3,5],
                                                [3,4,5,2,8,6,1,7,10]]))[0])
        # not ok: wrong dimensions of last row
        self.assertFalse(validateGrid(np.array([[5,3,4,6,7,8,9,1,2],
                                                [6,7,2,1,9,5,3,4,8],
                                                [1,9,8,3,4,2,5,6,7],
                                                [8,5,9,7,6,1,4,2,3],
                                                [4,2,6,8,5,3,7,9,1],
                                                [7,1,3,9,2,4,8,5,6],
                                                [9,6,1,5,3,7,2,8,4],
                                                [2,8,7,4,1,9,6,3,5],
                                                [3,4,5,2,8,6,1,7,9,2]]))[0])
        # not ok: there are 10 rows
        self.assertFalse(validateGrid(np.array([[5,3,4,6,7,8,9,1,2],
                                                [6,7,2,1,9,5,3,4,8],
                                                [1,9,8,3,4,2,5,6,7],
                                                [8,5,9,7,6,1,4,2,3],
                                                [4,2,6,8,5,3,7,9,1],
                                                [7,1,3,9,2,4,8,5,6],
                                                [9,6,1,5,3,7,2,8,4],
                                                [2,8,7,4,1,9,6,3,5],
                                                [3,4,5,2,8,6,1,7,9],
                                                [0,0,0,0,0,0,0,0,0]]))[0])
        
    # test that the sudoku object is created correctly
    def test_objectCreation(self):
        # not ok: the grid has a duplicated 9 in the next to last column
        obj = sudoku([[5,3,4,6,7,8,9,1,2],
                      [6,7,2,1,9,5,3,4,8],
                      [1,9,8,3,4,2,5,6,7],
                      [8,5,9,7,6,1,4,2,3],
                      [4,2,6,8,5,3,7,9,1],
                      [7,1,3,9,2,4,8,5,6],
                      [9,6,1,5,3,7,2,8,4],
                      [2,8,7,4,1,9,6,3,5],
                      [3,4,5,2,8,6,1,9,0]])
        self.assertFalse(hasattr(obj, 'grid'))      
        # ok
        obj = sudoku([[5,3,4,6,7,8,9,1,2],
                      [6,7,2,1,9,5,3,4,8],
                      [1,9,8,3,4,2,5,6,7],
                      [8,5,9,7,6,1,4,2,3],
                      [4,2,6,8,5,3,7,9,1],
                      [7,1,3,9,2,4,8,5,6],
                      [9,6,1,5,3,7,2,8,4],
                      [2,8,7,4,1,9,6,3,5],
                      [3,4,5,2,8,6,1,7,0]])
        # the object exists
        self.assertTrue(hasattr(obj, 'grid'))
        # the object is what we expect
        self.assertEqual(obj.grid, [[5,3,4,6,7,8,9,1,2],
                                    [6,7,2,1,9,5,3,4,8],
                                    [1,9,8,3,4,2,5,6,7],
                                    [8,5,9,7,6,1,4,2,3],
                                    [4,2,6,8,5,3,7,9,1],
                                    [7,1,3,9,2,4,8,5,6],
                                    [9,6,1,5,3,7,2,8,4],
                                    [2,8,7,4,1,9,6,3,5],
                                    [3,4,5,2,8,6,1,7,0]])
        
    # test we solve correctly
    def test_solver(self):
        obj = sudoku([[5,3,4,6,7,8,9,1,2],
                      [6,7,2,1,9,5,3,4,8],
                      [1,9,8,3,4,2,5,6,7],
                      [8,5,9,7,6,1,4,2,3],
                      [4,2,6,8,5,3,7,9,1],
                      [7,1,3,9,2,4,8,5,6],
                      [9,6,1,5,3,7,2,8,4],
                      [2,8,7,4,1,9,6,3,5],
                      [3,4,5,2,8,6,1,7,0]])
        obj.solve_puzzle()
        self.assertEqual(obj.grid, [[5,3,4,6,7,8,9,1,2],
                                    [6,7,2,1,9,5,3,4,8],
                                    [1,9,8,3,4,2,5,6,7],
                                    [8,5,9,7,6,1,4,2,3],
                                    [4,2,6,8,5,3,7,9,1],
                                    [7,1,3,9,2,4,8,5,6],
                                    [9,6,1,5,3,7,2,8,4],
                                    [2,8,7,4,1,9,6,3,5],
                                    [3,4,5,2,8,6,1,7,9]])
        
        obj = sudoku([[0,0,0,0,0,0,0,0,0],
                      [0,0,0,0,0,0,0,0,0],
                      [0,0,0,0,0,0,0,0,0],
                      [0,0,0,0,0,0,0,0,0],
                      [0,0,0,0,0,0,0,0,0],
                      [0,0,0,0,0,0,0,0,0],
                      [0,0,0,0,0,0,0,0,0],
                      [0,0,0,0,0,0,0,0,0],
                      [0,0,0,0,0,0,0,0,0]])
        obj.solve_puzzle()
        self.assertEqual(obj.grid, [[1,2,3,4,5,6,7,8,9], 
                                    [4,5,6,7,8,9,1,2,3],
                                    [7,8,9,1,2,3,4,5,6],
                                    [2,1,4,3,6,5,8,9,7],
                                    [3,6,5,8,9,7,2,1,4],
                                    [8,9,7,2,1,4,3,6,5],
                                    [5,3,1,6,4,2,9,7,8],
                                    [6,4,2,9,7,8,5,3,1],
                                    [9,7,8,5,3,1,6,4,2]])
        
        obj = sudoku([[1,2,3,4,5,6,7,8,9],
                      [4,5,6,7,8,9,1,2,3],
                      [7,8,9,1,2,3,4,5,6],
                      [2,1,4,3,6,5,8,9,7],
                      [3,6,5,8,9,7,2,1,4],
                      [8,9,7,2,1,4,3,6,5],
                      [5,3,1,6,4,2,9,7,8],
                      [6,4,2,9,7,8,5,3,1],
                      [9,7,8,5,3,1,6,4,2]])
        obj.solve_puzzle()
        self.assertEqual(obj.grid, [[1,2,3,4,5,6,7,8,9], 
                                    [4,5,6,7,8,9,1,2,3],
                                    [7,8,9,1,2,3,4,5,6],
                                    [2,1,4,3,6,5,8,9,7],
                                    [3,6,5,8,9,7,2,1,4],
                                    [8,9,7,2,1,4,3,6,5],
                                    [5,3,1,6,4,2,9,7,8],
                                    [6,4,2,9,7,8,5,3,1],
                                    [9,7,8,5,3,1,6,4,2]])
            
        
# buffer = True suppresses the print() outputs
unittest.main(argv=['first-arg-is-ignored'], exit=False, buffer=True)



......
----------------------------------------------------------------------
Ran 6 tests in 0.065s

OK
